In [1]:
# Fazendo o import dos pacotes que serão utilizados

import re
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from datetime import date
from datetime import datetime

from bs4 import BeautifulSoup
from docx import Document
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
# Criando um crawler que colherá o conteúdo da página desejada para que possamos coletar os dados

def crawl_website(url: str) -> str:
    
    try:
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        texto = soup.find(id="conteudo")
        paragrafo = ""
        for p in texto.find_all("p"):
            p = p.get_text()
            paragrafo += p
    except HTTPError as exc:
        print(exc)
    except URLError as exc:
        print(exc)
    else:
        return paragrafo

* Utilizamos a função re.compile para compilar um padrão de expressão regular. A ideia é usar essas expressões para correspondência posterior com o que queremos encontrar no Boletim da Sesab


In [4]:
REGEXP_CASOS = re.compile("([0-9.]+) casos confirmados")
REGEXP_MORTES = re.compile("([0-9.]+) tiveram óbito confirmado")
REGEXP_NOVOS_CASOS = re.compile("nas últimas 24 horas, foram registrados ([0-9.]+) casos de Covid-19")
REGEXP_NOVAS_MORTES = re.compile("e ([0-9.]+) óbit")
REGEXP_RECUPERADOS = re.compile("([0-9.]+) já são considerados recuperados") 
REGEXP_CASOS_ATIVOS = re.compile("([0-9.]+) encontram-se ativos")
REGEXP_CASOS_DESCARTADOS = re.compile("([0-9.]+) casos descartados")
REGEXP_CASOS_INVESTIGADOS = re.compile("([0-9.]+) em investigação")
REGEXP_VACINACAO_UMA_DOSE = re.compile("([0-9.]+) pessoas vacinadas com a primeira dose")
REGEXP_VACINACAO_DUAS_DOSES = re.compile("([0-9.]+) com a segunda dose ou dose única")
REGEXP_VACINACAO_DOSE_REFORCO = re.compile("([0-9.]+) com a dose de reforço")
REGEXP_VACINACAO_SEGUNDO_REFORCO = re.compile("([0-9.]+) com o segundo reforço")
REGEXP_VACINACAO_CRIANCAS = re.compile("([0-9.]+) crianças já foram imunizadas")
REGEXP_VACINACAO_CRIANCAS_SEGUNDA_DOSE = re.compile("([0-9.]+) já tomaram também a segunda dose")


* Agora vamos definir as datas que serão utilizadas para verificar se o boletim publicado pela Sesab é o mais atual e também para escrever o boletim posteriormente

In [5]:
DIAS = [
    'segunda',
    'terça',
    'quarta',
    'quinta',
    'sexta',
    'sábado',
    'domingo'
]

# Utilizaremos essa data para verificar se o boletim é o mais atual
data_atual = datetime.now()
data_em_texto = data_atual.strftime('%d/%m/%Y')

# Aqui, precisamos do dia da semana correspondente à data atual para escrever o boletim

indice_da_semana = data_atual.weekday()

dia_da_semana_por_extenso = DIAS[indice_da_semana]

if dia_da_semana_por_extenso == 'sábado' or dia_da_semana_por_extenso == 'domingo':
    dia = 'neste ' + dia_da_semana_por_extenso
else:
    dia = 'nesta ' + dia_da_semana_por_extenso

dia_da_semana = data_atual.day
    
# Utilizaremos esta ultima data para salvar o documento com a data do dia que ele foi escrito
data_do_arquivo = '{}-{}-{}'.format(data_atual.day, data_atual.month, data_atual.year)

* Vamos usar o pacote Selenium para navegar no portal onde são publicados os boletins da Sesab e para encontrar o boletim propriamente dito. 
Para o Selenium funcionar, é preciso baixar o webdriver correspondente a seu navagedor. No caso do Google Chrome, você pode obter o webdriver 
através desse link (https://chromedriver.chromium.org/downloads). É importante que esse arquivo esteja na mesma pasta em que estamos rodando esse script. 
Em seguinda, após abrirmos a página da Sesab, vamos usar a função find_element para encontrar o boletim. Veja aqui a documentação do Selenium 
(https://selenium-python.readthedocs.io/locating-elements.html)

* Aqui temos alguns padrões de título utilizados pela Sesab. Os boletins costumam conter as palavras 'casos ativos' ou 'Bahia registra', quando há casos registrados naquele dia. Em alguns momentos, quando não há registro de casos nas últimas 24 horas, o título do boletim pode conter as seguintes palavras: 'não registra' ou 'sem registro'.

* Aqui, vamos testar o primeiro link e o segundo, verificando, primeiro, se o título contém essas palavras e, posteriormente, se a data de publicação do boletim é a do dia atual.

In [7]:
navegador = webdriver.Chrome()
navegador.get('http://www.saude.ba.gov.br/category/emergencias-em-saude/')

titulo = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/div/div[2]/div[1]/div/div[2]/div/h2/a')
data_hora = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/div/div[2]/div[1]/div/div[2]/div/p[1]')

if ('casos ativos' or 'Bahia registra' in titulo.text) and data_em_texto in data_hora.text:
    print(titulo.text)
    print("O boletim saiu")
    titulo.click()
    get_url = navegador.current_url
elif 'não registra' in titulo.text and data_em_texto in data_hora.text:
    print(titulo.text)
    print("O boletim saiu")
    titulo.click()
    get_url = navegador.current_url
elif 'sem registro' in titulo.text and data_em_texto in data_hora.text:
    print(titulo.text)
    print("O boletim saiu")
    titulo.click()
    get_url = navegador.current_url
else:
    print('O primeiro link não é o boletim ou não é o boletim do dia')
    titulo = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/div/div[2]/div[2]/div/div[2]/div/h2/a')
    data_hora = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/div/div[2]/div[2]/div/div[2]/div/p[1]')
    
    if ('casos ativos' or 'Bahia registra' in titulo.text) and data_em_texto in data_hora.text:
        print(titulo.text)
        print("O boletim saiu")
        titulo.click()
        get_url = navegador.current_url
    elif 'não registra' in titulo.text and data_em_texto in data_hora.text:
        print(titulo.text)
        print("O boletim saiu")
        titulo.click()
        get_url = navegador.current_url
    elif 'sem registro' in titulo.text and data_em_texto in data_hora.text:
        print(titulo.text)
        print("O boletim saiu")
        titulo.click()
        get_url = navegador.current_url

    else:
        print('O segundo link também não é o boletim ou não é o boletim do dia')


Bahia registra 2.624 casos de Covid-19 e mais 5 óbitos
O boletim saiu


* Caso o boletim do dia tenha sido publicado, vamos coletar os dados.

In [8]:
conteudo = crawl_website(url=get_url)
casos = list(set(REGEXP_CASOS.findall(conteudo)))
mortes = list(set(REGEXP_MORTES.findall(conteudo)))
novos_casos = list(set(REGEXP_NOVOS_CASOS.findall(conteudo)))
novas_mortes = list(set(REGEXP_NOVAS_MORTES.findall(conteudo)))
recuperados = list(set(REGEXP_RECUPERADOS.findall(conteudo)))
casos_ativos = list(set(REGEXP_CASOS_ATIVOS.findall(conteudo)))
casos_descartados = list(set(REGEXP_CASOS_DESCARTADOS.findall(conteudo)))
casos_investigacao = list(set(REGEXP_CASOS_INVESTIGADOS.findall(conteudo)))
vacina_uma_dose = list(set(REGEXP_VACINACAO_UMA_DOSE.findall(conteudo)))
vacina_duas_doses = list(set(REGEXP_VACINACAO_DUAS_DOSES.findall(conteudo)))
vacina_reforco = list(set(REGEXP_VACINACAO_DOSE_REFORCO.findall(conteudo)))
vacina_segundo_reforco = list(set(REGEXP_VACINACAO_SEGUNDO_REFORCO.findall(conteudo)))
vacina_criancas = list(set(REGEXP_VACINACAO_CRIANCAS.findall(conteudo)))
vacina_criancas_segunda_dose = list(set(REGEXP_VACINACAO_CRIANCAS_SEGUNDA_DOSE.findall(conteudo)))

In [9]:
print(casos)
print(mortes)
print(novos_casos)
print(novas_mortes)
print(recuperados)
print(casos_ativos)
print(casos_descartados)
print(casos_investigacao)
print(vacina_uma_dose)
print(vacina_duas_doses)
print(vacina_reforco)
print(vacina_segundo_reforco)
print(vacina_criancas)
print(vacina_criancas_segunda_dose)

['1.570.045']
['30.024']
['2.624']
['5']
['1.533.075']
['6.946']
['1.907.228']
['342.097']
['11.622.227']
['10.686.928']
['6.292.700']
['650.670']
['968.998']
['550.674']


* Escrevendo o boletim com os dados coletados

In [10]:
document = Document()
document.add_paragraph(f'Opção de título 01: Bahia registra {novos_casos[0]} novos casos de Covid-19 e {novas_mortes[0]} mortes em 24 horas')
document.add_paragraph(f'Opção de subtítulo 01: O boletim epidemiológico divulgado pela Sesab {dia} ({dia_da_semana}) mostra que a Bahia atingiu {casos_ativos[0]} casos ativos da doença')
document.add_paragraph(f'Opção de título 02: Bahia registra {casos_ativos[0]} casos ativos de Covid-19 e {novas_mortes[0]} mortes pela doença')
document.add_paragraph(f'Opção de subtítulo 02: Ao todo, são {casos[0]} casos confirmados desde o início da pandemia')
document.add_paragraph(f'A Bahia registrou {novos_casos[0]} novos casos de Covid-19 e {novas_mortes[0]} mortes pela doença nas últimas 24 horas. Segundo o boletim epidemiológico da Secretaria de Saúde do Estado da Bahia (Sesab), divulgado {dia} ({dia_da_semana}), dos {casos[0]} casos confirmados desde o início da pandemia, {recuperados[0]} já são considerados recuperados e {mortes[0]} tiveram morte confirmada.')
document.add_paragraph(f'No estado, atualmente, são {casos_ativos[0]} casos ativos do novo coronavírus, além de {casos_descartados[0]} descartados e {casos_investigacao[0]} em investigação.')
document.add_paragraph('Vacinação')
document.add_paragraph(f'Até o momento temos {vacina_uma_dose[0]} pessoas vacinadas com a primeira dose, {vacina_duas_doses[0]} com a segunda dose ou dose única, {vacina_reforco[0]} com a dose de reforço e {vacina_segundo_reforco[0]} com o segundo reforço. Do público de 5 a 11 anos, {vacina_criancas[0]} crianças já foram imunizadas com a primeira dose e {vacina_criancas_segunda_dose[0]} já tomaram a segunda dose do imunizante.')

* Salvando o documento

In [11]:
document.save(f'boletim {data_do_arquivo}.doc')